### Packages

In [1]:
import numpy as np
import pandas as pd
import scipy
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
import statsmodels.graphics.tsaplots as sgt
import statsmodels.tsa.stattools as sts
#from statsmodels.tsa.arima_model import ARIMA
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
#from statsmodels.tsa import SARIMAX
from pmdarima.arima import auto_arima
from arch import arch_model
import yfinance
import warnings
warnings.filterwarnings("ignore")
sns.set()

### Getting the Data from yFinance

In [2]:
# Getting the current date
from datetime import date
today = date.today()
today

datetime.date(2021, 12, 29)

In [ ]:
raw_data = yfinance.download (tickers = "^BSESN ^NSEI ^FTSE ^N225 ^NYA", start = "2008-09-29", end = today,
                              interval = "1d", group_by = 'ticker', auto_adjust = True, treads = True)

In [ ]:
raw_data.tail()

### Creating the Data

In [ ]:
df_comp = raw_data.copy()

In [ ]:
df_comp['sensex'] = df_comp['^BSESN'].Close[:]
df_comp['nifty'] = df_comp['^NSEI'].Close[:]
df_comp['ftse'] = df_comp['^FTSE'].Close[:]
df_comp['nikkei'] = df_comp['^N225'].Close[:]
df_comp['nyse'] = df_comp['^NYA'].Close[:]

In [ ]:
df_comp = df_comp.iloc[1:]
del df_comp['^BSESN']
del df_comp['^NSEI']
del df_comp['^FTSE']
del df_comp['^N225']
del df_comp['^NYA']
df_comp=df_comp.asfreq('b')
df_comp=df_comp.fillna(method='ffill')

In [ ]:
df_comp.tail()

### Creating Returns

In [ ]:
df_comp['ret_sensex'] = df_comp.sensex.pct_change(1).mul(100)
df_comp['ret_nifty'] = df_comp.nifty.pct_change(1).mul(100)
df_comp['ret_ftse'] = df_comp.ftse.pct_change(1).mul(100)
df_comp['ret_nikkei'] = df_comp.nikkei.pct_change(1).mul(100)
df_comp['ret_nyse'] = df_comp.nyse.pct_change(1).mul(100)

In [ ]:
df_comp['norm_ret_sensex'] = df_comp.sensex.div(df_comp.ret_sensex[1])*100
df_comp['norm_ret_nifty'] = df_comp.nifty.div(df_comp.ret_nifty[1])*100
df_comp['norm_ret_ftse'] = df_comp.ret_ftse.div(df_comp.ret_ftse[1])*100
df_comp['norm_ret_nikkei'] = df_comp.ret_nikkei.div(df_comp.ret_nikkei[1])*100
df_comp['norm_ret_nyse'] = df_comp.ret_nyse.div(df_comp.ret_nyse[1])*100

In [ ]:
df_comp.tail()

### Splitting the Data

In [ ]:
size = int(len(df_comp)*0.8)
df, df_test = df_comp.iloc[:size], df_comp.iloc[size:]

In [ ]:
df_test.head()

### Fitting the Model

### ARMAX

In [ ]:
start_date = "2019-01-01"
end_date = "2021-12-10"
model_ret_armax = ARIMA(df.ret_sensex[1:], exog = df[["ret_ftse","ret_nikkei","ret_nyse"]][1:], order = (1,0,1))

results_ret_armax = model_ret_armax.fit()

df_pred_armax = results_ret_armax.predict(start = start_date, end = end_date, 
                                          exog = df_test[["ret_ftse","ret_nikkei","ret_nyse"]][start_date:end_date]) 

df_pred_armax[start_date:end_date].plot(figsize = (20,5), color = "red")
df_test.ret_sensex[start_date:end_date].plot(color = "blue")
plt.title("Predictions vs Actual (Returns)", size = 24)
plt.show()

In [ ]:
# Lets zoom in a bit

start_date = "2019-04-01"
end_date = "2020-07-01"
model_ret_armax = ARIMA(df.ret_sensex[1:], exog = df[["ret_ftse","ret_nikkei","ret_nyse"]][1:], order = (1,0,1))

results_ret_armax = model_ret_armax.fit()

df_pred_armax = results_ret_armax.predict(start = start_date, end = end_date, 
                                          exog = df_test[["ret_ftse","ret_nikkei","ret_nyse"]][start_date:end_date]) 

df_pred_armax[start_date:end_date].plot(figsize = (20,5), color = "red")
df_test.ret_sensex[start_date:end_date].plot(color = "blue")
plt.title("Predictions vs Actual (Returns)", size = 24)
plt.show()

### SARIMAX

In [ ]:
start_date = "2019-04-01"
end_date = "2020-07-01"
model_ret_sarimax = SARIMAX(df.ret_sensex[1:], exog = df[["ret_ftse","ret_nikkei","ret_nyse"]][1:], 
                            order = (3,0,4), seasonal_order = (3,0,2,5))
results_ret_sarimax = model_ret_sarimax.fit()

df_pred_sarimax = results_ret_sarimax.predict(start = start_date, end = end_date, 
                                              exog = df_test[["ret_ftse","ret_nikkei","ret_nyse"]][start_date:end_date]) 

df_pred_sarimax[start_date:end_date].plot(figsize = (20,5), color = "red")
df_test.ret_sensex[start_date:end_date].plot(color = "blue")
plt.title("Predictions vs Actual", size = 24)
plt.show()

### Auto ARIMA

In [ ]:
# model_auto = auto_arima(df.ret_sensex[1:], exogenous = df[['ret_ftse','ret_nikkei','ret_nyse']][1:],
#                        m = 5, max_p = 5, max_q = 5, max_P = 5, max_Q = 5)

In [ ]:
# df_auto_pred = pd.DataFrame(model_auto.predict(n_periods = len(df_test[start_date:end_date]),
#                             exogenous = df_test[['ret_ftse','ret_nikkei','ret_nyse']][start_date:end_date]),
#                             index = df_test[start_date:end_date].index)

In [ ]:
# df_auto_pred.plot(figsize = (20,5), color = "red")
# df_test.ret_sensex[start_date:end_date].plot(color = "blue")
# plt.title("Auto Model Predictions vs Real Data", size = 24)
# plt.show()

### FB Prophet Model for future predictions

In [ ]:
df_comp.head()

In [ ]:
df_comp = df_comp.iloc[1:]

In [ ]:
df_comp.head()

In [ ]:
df_nifty = df_comp["nifty"]

In [ ]:
df_nifty.head()

In [ ]:
df_nifty.reset_index()

In [ ]:
df_nifty.columns = ['ds','y']

In [ ]:
df_nifty.reset_index()

In [ ]:
df_nifty.columns = ['DS', 'Y']

In [ ]:
df_nifty.columns

In [ ]:
from fbprophet import Prophet

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
m = Prophet(
    growth="linear",
    #holidays=holidays,
    #seasonality_mode="multiplicative",
    changepoint_prior_scale=30,
    seasonality_prior_scale=35,
    ###cap=3.00,
    ###floor=.65*125,
    holidays_prior_scale=20,
    daily_seasonality=False,
    weekly_seasonality=False,
    yearly_seasonality=False,
    ).add_seasonality(
        name='monthly',
        period=30.5,
        fourier_order=55
    ).add_seasonality(
        name='daily',
        period=1,
        fourier_order=15
    ).add_seasonality(
        name='weekly',
        period=7,
        fourier_order=20
    ).add_seasonality(
        name='yearly',
        period=365.25,
        fourier_order=20
    ).add_seasonality(
        name='quarterly',
        period=365.25/4,
        fourier_order=5,
        prior_scale=15)

In [ ]:
m.fit(df_nifty)